# Load Libraries, Modules, and Magics

In [71]:
import pandas as pd
import pandas_profiling as pdp
import numpy as np
import random
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV, f_classif
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, normalize, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn import metrics
import xgboost as xgb
import operator
from scipy.stats import uniform as sp_rand
from scipy.stats import randint as sp_randint
import scipy.stats as st
from scipy.stats.mstats import mode
from matplotlib import pyplot as plt
import seaborn as sns
import time
import math
import datetime
import random
from yellowbrick.features import ParallelCoordinates, Rank2D, JointPlotVisualizer
from yellowbrick.regressor import ResidualsPlot, AlphaSelection, PredictionError

%matplotlib inline

# Load (fixed) Data files

In [72]:
train_orig = pd.read_csv('./assets/train.csv', header=0)
test_orig = pd.read_csv('./assets/test.csv', header=0)
severity_type = pd.read_csv('./assets/severity_type.csv', header=0)
log_feature = pd.read_csv('./assets/log_feature.csv', header=0)
event_type = pd.read_csv('./assets/event_type_fixed.csv', header=0)
resource_type = pd.read_csv('./assets/resource_type_fixed.csv', header=0)

# Functions That Make EDA Easier

In [73]:
def norm_rows(df):
    with np.errstate(invalid='ignore'):
        return df.div(df.sum(axis=1), axis=0).fillna(0)

In [74]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

In [75]:
def print_value_counts(input_df,col_name,is_normalized=False):
    for value in input_df[col_name].unique():
        print(value)
        print(input_df.loc[input_df[col_name] == value].fault_severity.value_counts(normalize=is_normalized))

In [76]:
def print_value_counts_spec(input_df,col_name,col_value,is_normalized=False):
    print(col_value)
    print(input_df.loc[input_df[col_name] == col_value].fault_severity.value_counts(normalize=is_normalized))

In [77]:
def convert_strings_to_ints(input_df,col_name,output_col_name):
    labels, levels = pd.factorize(input_df[col_name])
    input_df[output_col_name] = labels
    output_dict = dict(zip(input_df[col_name],input_df[output_col_name]))
    return (output_dict,input_df)

In [78]:
#assumes row normalized, doesn't do eps thing
def get_log_loss_row(row):
    ans = row['fault_severity']
    if (ans == 0):
        return -1.0 * np.log(row['predict_0'])
    elif (ans == 1):
        return -1.0 * np.log(row['predict_1'])
    elif (ans == 2):
        return -1.0 * np.log(row['predict_2'])
    else:
        print('not_acceptable_value')
        raise ValueError('Not of correct class')
        return -1000

In [79]:
def get_log_loss_row_two_classes(row):
    ans = row['fault_severity']
    if (ans == 0):
        return -1.0 * np.log(row['predict_low'])
    elif (ans == 1):
        return -1.0 * np.log(row['predict_high'])
    else:
        print('not_acceptable_value')
        raise ValueError('Not of correct class')
        return -1000

# EDA

In [80]:
pdp.ProfileReport(train_orig)

Number of variables,3
Number of observations,7381
Total Missing (%),0.0%
Total size in memory,173.1 KiB
Average record size in memory,24.0 B
Numeric,2
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [81]:
pdp.ProfileReport(test_orig)

Number of variables,2
Number of observations,11171
Total Missing (%),0.0%
Total size in memory,174.6 KiB
Average record size in memory,16.0 B
Numeric,1
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [82]:
pdp.ProfileReport(severity_type)

Number of variables,2
Number of observations,18552
Total Missing (%),0.0%
Total size in memory,290.0 KiB
Average record size in memory,16.0 B
Numeric,1
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [83]:
pdp.ProfileReport(log_feature)

Number of variables,3
Number of observations,58671
Total Missing (%),0.0%
Total size in memory,1.3 MiB
Average record size in memory,24.0 B
Numeric,2
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [84]:
pdp.ProfileReport(resource_type)

Number of variables,2
Number of observations,22877
Total Missing (%),0.0%
Total size in memory,357.5 KiB
Average record size in memory,16.0 B
Numeric,1
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [85]:
pdp.ProfileReport(event_type)

Number of variables,2
Number of observations,34083
Total Missing (%),0.0%
Total size in memory,532.6 KiB
Average record size in memory,16.0 B
Numeric,1
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,0


### Analysis:
The data files present multiple views into linked data tables. Based on cardinality, my educated guess is that the ID field  is the key between the tables, with location and log_feature containing relevant outage data.

# Feature Engineering

## Join the tables into one monolithic file to prepare for modeling, including test and train. I'm not sure why or how they were seperated, but I don't trust that they were separated without bias.

In [86]:
df = pd.concat([train_orig,test_orig], ignore_index=True)
df.head()

C:\Users\jerome\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,fault_severity,id,location
0,1.0,14121,location 118
1,0.0,9320,location 91
2,1.0,14394,location 152
3,1.0,8218,location 931
4,0.0,14804,location 120


In [87]:
df.shape

(18552, 3)

In [88]:
# Let's get a look at the breakdown of fault_severity, since this will be our target vector
df.fault_severity.value_counts()

0.0    4784
1.0    1871
2.0     726
Name: fault_severity, dtype: int64

In [89]:
# now merge the event_type, key on id
event_type = pd.merge(df, event_type, on='id')
event_type.head()

,fault_severity,id,location,event_type
0,1.0,14121,location 118,event_type 34
1,1.0,14121,location 118,event_type 35
2,0.0,9320,location 91,event_type 34
3,0.0,9320,location 91,event_type 35
4,0.0,9320,location 91,event_type 34


In [90]:
# ...and get the unique event types
event_type_unq = pd.DataFrame(event_type['event_type'].value_counts())
event_type_unq.head()

,event_type
event_type 11,9041
event_type 35,6814
event_type 34,6116
event_type 15,5003
event_type 20,1594


In [92]:
# these are the top 5 event types.

In [95]:
# now let's deal with the nulls in event_type
event_type.isnull().sum()

fault_severity    20437
id                    0
location              0
event_type            0
dtype: int64

In [105]:
event_type[event_type.fault_severity==2].count()

fault_severity    1428
id                1428
location          1428
event_type        1428
dtype: int64

In [106]:
event_type[event_type.isnull().any(axis=1)]

,fault_severity,id,location,event_type
13645,NaN,11066,location 481,event_type 35
13646,NaN,11066,location 481,event_type 34
13647,NaN,18000,location 962,event_type 11
13648,NaN,18000,location 962,event_type 15
13649,NaN,18000,location 962,event_type 11
13650,NaN,18000,location 962,event_type 15
13651,NaN,16964,location 491,event_type 35
13652,NaN,16964,location 491,event_type 34
13653,NaN,4795,location 532,event_type 27
13654,NaN,4795,location 532,event_type 10
